In [46]:
%store -r __importData
import pandas as pd
import numpy as np
import ggplot as gplt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import time
import sys

In [2]:
__importData

import group_table


C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


import airline_table


C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


import order_table
import train_set
import test_set


In [38]:
gb_subline = group_table.groupby("sub_line", as_index=False).agg({"price": {"subline_price_mean": "mean",
                                                                            "subline_price_std": "std"},
                                                                  "days": {"subline_days_mean": "mean",
                                                                           "subline_days_std": "std"}})
gb_subline.columns = [gb_subline.columns.values[0][0]] + [x[1] for x in gb_subline.columns.values[1:]]

gb_area = group_table.groupby("area", as_index=False).agg({"price": {"area_price_mean": "mean",
                                                                     "area_price_std": "std"},
                                                           "days": {"area_days_mean": "mean",
                                                                    "area_days_std": "std"}})
gb_area.columns = [gb_area.columns.values[0][0]] + [x[1] for x in gb_area.columns.values[1:]]

gb_overall = group_table.groupby(["sub_line", "area"]).agg({"price": {"overall_price_mean": "mean",
                                                                      "overall_price_std": "std"},
                                                            "days": {"overall_days_mean": "mean",
                                                                     "overall_days_std": "std"}}).reset_index()
gb_overall.columns = [x[0] for x in gb_overall.columns.values[:2]] + [x[1] for x in gb_overall.columns.values[2:]]

for table in [gb_subline, gb_area, gb_overall]:
    table.fillna(-999, inplace=True)

C:\Users\S\Anaconda3\lib\site-packages\pandas\core\groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [39]:
tmp_group = pd.merge(group_table, gb_subline, on="sub_line", how="left")
tmp_group = pd.merge(tmp_group, gb_area, on="area", how="left")
tmp_group = pd.merge(tmp_group, gb_overall, on=["sub_line", "area"], how="left")

In [47]:
for view in ["subline", "area", "overall"]:
    group_table[view + "_price_z"] = (tmp_group["price"] - tmp_group[view + "_price_mean"]) / (tmp_group[view + "_price_std"] + \
                                                                                              sys.float_info.epsilon)
    group_table[view + "_days_z"] = (tmp_group["days"] - tmp_group[view + "_days_mean"]) / (tmp_group[view + "_days_std"] + \
                                                                                            sys.float_info.epsilon)

In [51]:
group_table[["group_id"] + list(group_table.columns[-6:])].to_csv("group_days_price.csv", encoding="utf-8", index=False)